#### Library imports / L5Kit setup

In [3]:
# PYTHON VERSION SHOULD BE 3.8.8
import sys
print(sys.version)

# NUMPY VERSION SHOULD BE 1.19.0
import numpy as np
np.version.version

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


'1.19.0'

In [5]:
import matplotlib.pyplot as plt

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data

import os

In [6]:
# opening the zarr_dataset
# os.environ["L5KIT_DATA_FOLDER"] = "../../prediction-dataset/"
os.environ["L5KIT_DATA_FOLDER"] = "../prediction-sample-dataset/"
cfg = load_config_data("./visualisation_config.yaml")
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()

In [7]:
# using EgoDataset interface to extract AV data
rast = build_rasterizer(cfg, dm)
ego_dataset = EgoDataset(cfg, zarr_dataset, rast)

In [8]:
scene_frame_indices = []
scene_frame_indices = zarr_dataset.scenes[:]["frame_index_interval"]

#### Spline functions

In [ ]:
def linearRegression(positions, window_size):
    """
    Use linear regression on the inputted positions data to predict the position in the next timestep

    positions -- 2D-array of the position data in the format [[x0, y0], [x1,y1], ...]
    window_size -- describes how many of the latest data points to use to predict the next timestep
    """
    predictions = []
    for i in range(window_size, len(positions) - 1):
        # use linear regression to predict next x-position
        # use linear regression to predict next y-position
        # combine results into predictions array
        pass
    return predictions

In [ ]:
def multivariateRegression(positions):
    pass

# timeseries
# p auto regressive
# CI

#### Plotting functions

In [ ]:
def plotData(positions, predictions):
    
    positions_x = [position[0] for position in positions]
    positions_y = [position[1] for position in positions]

    predictions_x = [position[0] for position in predictions]
    predictions_y = [position[1] for position in predictions]

    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    ax = plt.subplot(1, 2, 1)
    ax.scatter(positions_x, positions_y, label="Positions", s=5, alpha=1, color='c')
    ax.legend()
    ax.set_title("Position Data")
    ax.set_xlabel("Position (m)")
    ax.set_ylabel("Position (m)")

    ax1 = plt.subplot(1, 3, 2)
    ax1.scatter(positions_x, positions_y, label="Positions", s=5, alpha=0.6, color='c')
    ax1.scatter(predictions_x, predictions_y, label="Predictions", s=5, alpha=0.4, color='m')
    ax1.legend()
    ax1.set_title("Basic Kalman Filter")
    
    # https://stackoverflow.com/questions/38855748/matplotlib-generating-strange-y-axis-on-certain-data-sets
    ax.get_yaxis().get_major_formatter().set_useOffset(False)
    ax1.get_yaxis().get_major_formatter().set_useOffset(False)

    plt.tight_layout()
    plt.show()

In [11]:
def compareFilters(scene_number):
    positions = zarr_dataset.frames[slice(scene_frame_indices[scene_number][0], scene_frame_indices[scene_number][1])]["ego_translation"]
    positions = [[pos[0], pos[1]] for pos in positions]
    print(positions)
    predictions = linearRegression(positions)
    
    #plotData(positions, predictions)

In [13]:
# compareFilters(0)